# Sentiment analysis with Word2Vec

### Exercise objectives:
- Convert words to vectors with Word2Vec
- Use the word representation given by Word2vec to feed a RNN

<hr>

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [1]:
!pip freeze | grep gensim

gensim==4.2.0


# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [2]:
import numpy as np

In [3]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)

    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)

        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]

        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]

    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2024-08-15 15:20:18.925932: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "INVALID_ARGUMENT: Error executing an HTTP request: HTTP response code 400 with body '{"error":"invalid_grant","error_description":"Invalid grant: account not found"}'". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNJ6KUO/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNJ6KUO/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2024-08-15 15:20:42.916601: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In the previous exercise, you trained a Word2vec representation and converted all your training sentences in order to feed them into a RNN, as shown in the first step of this Figure: 

<img src="word2vec_representation.png" width="400px" />



❓ **Question** ❓ Here, let's re-do exactly what you have done in the previous exercise. First, train a word2vec model (with the arguments that you want) on your training sentence. Store it into the `word2vec` variable.

In [4]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train,min_count=5, vector_size=200)

Let's reuse the functions of the previous exercise to convert your training and test data into something you can feed into a RNN.

❓ **Question** ❓ Read the following function to be sure you understand what is going on, and run it.

In [5]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])

    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []

    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)

    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

☝️ To be sure that it worked, let's check the following for `X_train_pad` and `X_test_pad`:
- they are numpy arrays
- they are 3-dimensional
- the last dimension is of the size of your word2vec embedding space (you can get it with `word2vec.wv.vector_size`
- the first dimension is of the size of your `X_train` and `X_test`

✅ **Good Practice** ✅ Such tests are quite important! Not only in this exercise, but in real-life applications. It prevents from finding errors too late and from letting them propagate through the entire notebook.

In [6]:
# TEST ME
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)

# Baseline model

It is always good to have a very simple model to test your own model against - to be sure you are doing something better than a very simple algorithm.

❓ **Question** ❓ What is your baseline accuracy? In this case, your baseline can be to predict the label that is the most present in `y_train` (of course, if the dataset is balanced, the baseline accuracy is 1/n where n is the number of classes - 2 here).

In [20]:
count_1 = np.sum(y_train==1)
count_0 = np.sum(y_train==0)

print(count_1,count_0)

baseline_score = 0.5

1235 1265


In [38]:
from tensorflow.keras.layers import Normalization, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# The model

❓ **Question** ❓ Write a RNN with the following layers:
- a `Masking` layer
- a `LSTM` with 20 units and `tanh` activation function
- a `Dense` with 10 units
- an output layer that depends on your task

Then, compile your model (we advise you to use the `rmsprop` as the optimizer - at least to begin with)

In [25]:
def create_model_lstm():

    model = Sequential()

    model.add(layers.Normalization(input_shape=(200,200)))

    # masking layer to ignore padded values
    model.add(layers.Masking(mask_value=0.0))

    # adding RNN layer
    model.add(layers.LSTM(units=20, activation = 'tanh',use_bias=True))

    #dense layer with 10 neurons
    model.add(layers.Dense(10, activation='relu'))

    # dense layer specific to the task of predicting a salary
    model.add(layers.Dense(1, activation='sigmoid'))

    #compiling the model
    model.compile(
        loss = 'binary_crossentropy',
        optimizer = 'rmsprop',
        metrics = ['accuracy']
    )

    return model

In [26]:
model = create_model_lstm()

❓ **Question** ❓ Fit the model on your embedded and padded data - do not forget the early stopping criterion.

❗ **Remark** ❗ Your accuracy will greatly depend on your training corpus. Here just make sure that your performance is above the baseline model (which should be the case even if you loaded only 20% of the initial IMDB data).

In [28]:
es = EarlyStopping(patience = 5, restore_best_weights=True)

history = model.fit(
     X_train_pad,
     y_train,
     validation_split = 0.2,
     batch_size = 32,
     epochs = 30,
     callbacks = [es]
)

Epoch 1/30
63/63 [==============================] - 2s 31ms/step - loss: 0.5973 - accuracy: 0.6800 - val_loss: 0.6632 - val_accuracy: 0.6220
Epoch 2/30
63/63 [==============================] - 2s 31ms/step - loss: 0.5944 - accuracy: 0.6820 - val_loss: 0.6506 - val_accuracy: 0.6180
Epoch 3/30
63/63 [==============================] - 2s 31ms/step - loss: 0.5901 - accuracy: 0.6890 - val_loss: 0.6220 - val_accuracy: 0.6400
Epoch 4/30
63/63 [==============================] - 2s 31ms/step - loss: 0.5749 - accuracy: 0.7040 - val_loss: 0.6184 - val_accuracy: 0.6680
Epoch 5/30
63/63 [==============================] - 2s 31ms/step - loss: 0.5647 - accuracy: 0.7095 - val_loss: 0.6325 - val_accuracy: 0.6400
Epoch 6/30
63/63 [==============================] - 2s 31ms/step - loss: 0.5535 - accuracy: 0.7215 - val_loss: 0.6583 - val_accuracy: 0.6460
Epoch 7/30
63/63 [==============================] - 2s 31ms/step - loss: 0.5487 - accuracy: 0.7285 - val_loss: 0.7008 - val_accuracy: 0.6320
Epoch 8/30
63

❓ **Question** ❓ Evaluate your model on the test set

In [29]:
results = model.evaluate(X_test_pad,y_test)

79/79 [==============================] - 1s 9ms/step - loss: 0.6367 - accuracy: 0.6412


In [30]:
accuracy = results[1]

print(accuracy)

0.6412000060081482


# Trained Word2Vec - Transfer Learning

Your accuracy, while above the baseline model, might be quite low. There are multiple options to improve it, as data cleaning and improving the quality of the embedding.

We won't dig into data cleaning strategies here. Let's try to improve the quality of our embedding. But instead of just loading a larger corpus, why not benefiting from the embedding that others have learned? Because, the quality of an embedding, i.e. the proximity of the words, can be derived from different tasks. This is exactly what transfer learning is.



❓ **Question** ❓ List all the different models available in the word2vec thanks to this: 

In [31]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


ℹ️ You can also find the list of the models and their size on the [`gensim-data` repository](https://github.com/RaRe-Technologies/gensim-data#models).

❓ **Question** ❓ Load one of the pre-trained word2vec embedding spaces. 

You can do that with `api.load(the-model-of-your-choice)`, and store it in `word2vec_transfer`

<details>
    <summary>💡 Hint</summary>
    
The `glove-wiki-gigaword-50` model is a good candidate to start with as it is smaller (65 MB).

</details>

In [32]:
word2vec_transfer = api.load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded


❓ **Question** ❓ Check the size of the vocabulary, but also the size of the embedding space.

In [36]:
vocab_size = len(word2vec_transfer)

❓ Let's embed `X_train` and `X_test`, same as in the first question where we provided the functions to do so! (There is a slight difference in the `embed_sentence_with_TF` function that we will not dig into)

In [37]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])

    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []

    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)

    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

❓ **Question** ❓  Do not forget to pad your results and store it in `X_train_pad_2` and `X_test_pad_2`.

In [39]:
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='pre')
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='pre')

❓ **Question** ❓ Reinitialize a model and fit it on your new embedded (and padded) data!  Evaluate it on your test set and compare it to your previous accuracy.

❗ **Remark** ❗ The training here could take some time. You can just compute 10 epochs (this is **not** a good practice, it is just not to wait too long) and go to the next exercise while it trains - or take a break, you probably deserve it ;)

In [44]:
def create_model_lstm_2():

    model = Sequential()

    model.add(layers.Normalization(input_shape=X_train_pad_2.shape[1:]))

    # masking layer to ignore padded values
    model.add(layers.Masking(mask_value=0.0))

    # adding RNN layer
    model.add(layers.LSTM(units=20, activation = 'tanh',use_bias=True))

    #dense layer with 10 neurons
    model.add(layers.Dense(10, activation='relu'))

    # dense layer specific to the task of predicting a salary
    model.add(layers.Dense(1, activation='sigmoid'))

    #compiling the model
    model.compile(
        loss = 'binary_crossentropy',
        optimizer = 'rmsprop',
        metrics = ['accuracy']
    )

    return model

In [43]:
X_train_pad_2.shape[1:]

(1122, 50)

In [45]:
model_2 = create_model_lstm_2()

In [46]:
es = EarlyStopping(patience = 5, restore_best_weights=True)

history_2 = model_2.fit(
     X_train_pad_2,
     y_train,
     validation_split = 0.2,
     batch_size = 32,
     epochs = 30,
     callbacks = [es]
)

Epoch 1/30
63/63 [==============================] - 11s 144ms/step - loss: 0.6906 - accuracy: 0.5405 - val_loss: 0.6856 - val_accuracy: 0.5420
Epoch 2/30
63/63 [==============================] - 8s 132ms/step - loss: 0.6751 - accuracy: 0.5840 - val_loss: 0.6672 - val_accuracy: 0.5980
Epoch 3/30
63/63 [==============================] - 8s 133ms/step - loss: 0.6497 - accuracy: 0.6320 - val_loss: 0.6286 - val_accuracy: 0.6540
Epoch 4/30
63/63 [==============================] - 8s 133ms/step - loss: 0.6161 - accuracy: 0.6760 - val_loss: 0.6301 - val_accuracy: 0.6520
Epoch 5/30
63/63 [==============================] - 8s 132ms/step - loss: 0.5938 - accuracy: 0.6905 - val_loss: 0.5958 - val_accuracy: 0.6900
Epoch 6/30
63/63 [==============================] - 8s 135ms/step - loss: 0.5795 - accuracy: 0.7020 - val_loss: 0.5774 - val_accuracy: 0.7040
Epoch 7/30
63/63 [==============================] - 8s 133ms/step - loss: 0.5708 - accuracy: 0.7095 - val_loss: 0.5991 - val_accuracy: 0.6840
Epoch

In [48]:
res = model_2.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 76.720%


Because your new word2vec has been trained on a large corpus, it has a representation for many many words! Way more than with your small dataset, especially as you discarded words that were not present more than a given number of times in the train set. For that reason, you have way more embedded words in your train and test set, which makes each iteration longer than previously